In [47]:
from langgraph.graph import StateGraph, START, END
from langchain_groq import ChatGroq
from typing import TypedDict, Literal
from pydantic import BaseModel, Field
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
load_dotenv()
model = ChatGroq(model="llama-3.3-70b-versatile")


In [48]:
class OutputSchema(BaseModel):
    sentiment: str = Field(..., description="The sentiment of the text, either 'positive' or 'negative'")

In [ ]:
class ReviewSchema(TypedDict):
    
    sentiment: str
    review: str
    deescription: str
    response: str

In [ ]:
#
strmodel = model.with_structured_output(OutputSchema)

In [51]:
review = """I absolutely loved the new design of your website! It's so user-friendly and visually appealing. Great job!"""

In [52]:
def conditional_workflow(state: ReviewSchema) -> Literal["positive","negative"]:
    result = state["review"]
    sentiment = strmodel.invoke(result).sentiment
    if sentiment == "positive":
        return 'positive'
    else:
        return 'negative'

In [53]:
def daigonosis(state: ReviewSchema):
    result = f'find the sentiment of the review {state["review"]}'
    sentiment = strmodel.invoke(result).sentiment
    return {"sentiment": sentiment}

In [ ]:
def positive(state: ReviewSchema):
    prompt = f'For the Following review: {state["review"]}, write a response and say user to do feedback on website also.'
    response_msg = model.invoke(prompt)
    
    return {"response": response_msg.content}

In [ ]:
def negative(state: ReviewSchema):
    prompt = f'For the Following review: {state["review"]}, write a apology response and offer discount coupon for next purchase.'

    response_msg = model.invoke(prompt)
    
    return {"response": response_msg.content}

In [ ]:
graph = StateGraph(ReviewSchema)
graph.add_node("diagonosis", daigonosis)
graph.add_node("positive", positive)
graph.add_node("negative", negative)    
graph.add_edge(START, "diagonosis")
graph.add_conditional_edges(
    "diagonosis", 
    conditional_workflow,
    {
        "positive": "positive",  
        "negative": "negative"   
    }
)
graph.add_edge("positive", END)
graph.add_edge("negative", END)
workflow = graph.compile()

Workflow compiled successfully!


In [57]:
# Invoke the workflow with the review
result = workflow.invoke({"review": review})
print("\n--- Results ---")
print(f"Sentiment: {result.get('sentiment')}")
print(f"Response: {result.get('response')}")


--- Results ---
Sentiment: positive
Response: Dear valued customer,

Thank you so much for taking the time to share your thoughts on our newly designed website! We're thrilled to hear that you find it user-friendly and visually appealing. Our team worked hard to create a seamless and engaging experience for our visitors, and it's wonderful to know that we've been able to achieve that.

We're always looking for ways to improve and would love to hear more of your thoughts on our website. If you have any feedback or suggestions on how we can further enhance your experience, please don't hesitate to let us know. Your input is invaluable to us, and we'd be grateful if you could take a moment to share your thoughts on the following:

* What do you like most about the new design?
* Are there any features or functionalities that you'd like to see added?
* Are there any areas where you think we could improve?

You can provide your feedback by replying to this email or by visiting our website a